In [ ]:
import os
import shutil
import json
import yaml

In [ ]:
path_to_perf_co3d = "/home/yoonwoo/data/perfception/perfception_co3d"

In [ ]:
cnt = 0
co3d_lists = os.listdir(path_to_perf_co3d)
for co3d_dir_name in co3d_lists:
    co3d_dir_path = os.path.join(path_to_perf_co3d, co3d_dir_name)
    results_path = os.path.join(co3d_dir_path, "results.json")
    if not os.path.exists(results_path):
        # shutil.rmtree(co3d_dir_path)
        cnt += 1
print(cnt, len(co3d_lists))

In [ ]:
with open("../../dataloader/co3d_lists/co3d_list.json") as fp:
    co3d_all_lists = json.load(fp)

co3d_lists = os.listdir(path_to_perf_co3d)
print(co3d_lists[0], list(co3d_all_lists.keys())[0])
co3d_missing_scene_list = []
for co3d_dir_name in co3d_all_lists.keys():
    if "plenoxel_co3d_" + co3d_dir_name not in co3d_lists:
        co3d_missing_scene_list.append(co3d_dir_name)

def generate_sweep_co3d():
    params = dict()
    params["scene_name"] = dict(values=co3d_missing_scene_list)
    params["entity"] = dict(value="postech_cvlab")
    config = dict()
    config["method"] = "grid"
    config["program"] = "run.py"
    config["parameters"] = params
    with open("../../cache/co3d_missing.yaml", "w") as fp:
        yaml.dump(config, fp)

generate_sweep_co3d()

# Find missing logs for rendering

In [22]:
import os
import json
import yaml
import shutil

In [30]:
rendered_data_path = "/home/yoonwoo/data/render"
assert os.path.exists(rendered_data_path)
with open("../../dataloader/co3d_lists/co3d_list.json") as fp:
    co3d_all_lists = json.load(fp)

co3d_missing_scene_list = []
for (scene_number, class_name) in co3d_all_lists.items():
    dirpath = os.path.join(rendered_data_path, class_name, scene_number)
    bgpath = os.path.join(dirpath, "bg")
    if not os.path.exists(os.path.join(bgpath, "image049.jpg")): 
        co3d_missing_scene_list.append(scene_number)
        # if os.path.exists(dirpath):
        #     shutil.rmtree(dirpath)
        print(bgpath)

In [20]:
def generate_sweep_co3d():
    params = dict()
    params["scene_name"] = dict(values=co3d_missing_scene_list)
    params["entity"] = dict(value="postech_cvlab")
    config = dict()
    config["method"] = "grid"
    config["program"] = "run.py"
    config["parameters"] = params
    with open("../../cache/co3d_missing.yaml", "w") as fp:
        yaml.dump(config, fp)

generate_sweep_co3d()

In [29]:
# Move the rendered results back
import os
rendered_dir = "../../render"
for cls_name in os.listdir(rendered_dir):
    cls_path = os.path.join(rendered_dir, cls_name) 
    for scene_num in os.listdir(cls_path):
        scene_path = os.path.join(cls_path, scene_num)
        scene_mv_to = os.path.join(rendered_data_path, cls_name, scene_num)
        shutil.copytree(scene_path, scene_mv_to)

In [48]:
# Merge the bg and fg/bg
import os

co3d_2d_orig = "/home/yoonwoo/data/perfception/perfception_co3d_2d"
assert os.path.exists(co3d_2d_orig)

for cls_name in os.listdir(co3d_2d_orig):
    cls_path = os.path.join(co3d_2d_orig, cls_name)
    for scene_num in os.listdir(cls_path):
        scene_path = os.path.join(cls_path, scene_num)
        bg_path = os.path.join(scene_path, "bg")
        # if os.path.exists(bg_path): 
        #     shutil.rmtree(bg_path)

for cls_name in os.listdir(co3d_2d_orig):
    cls_path = os.path.join(co3d_2d_orig, cls_name)
    for scene_num in os.listdir(cls_path): 
        scene_path = os.path.join(cls_path, scene_num)
        scene_path_src = os.path.join("/home/yoonwoo/data/render", cls_name, scene_num)
        pose_path = os.path.join(scene_path_src, "poses.npy")
        intrinsics_path = os.path.join(scene_path_src, "intrinsics.npy")
        bg_path = os.path.join(scene_path_src, "bg")
        assert os.path.exists(pose_path)
        assert os.path.exists(intrinsics_path)
        assert os.path.exists(bg_path)
        assert os.path.exists(scene_path)
        shutil.move(pose_path, os.path.join(scene_path, "poses.npy"))
        shutil.move(intrinsics_path, os.path.join(scene_path, "intrinsics.npy"))
        shutil.move(bg_path, os.path.join(scene_path, "bg"))